## 回调函数

什么是回调函数
当程序运行是，一般情况下，应用程序会时常通过API调用库里所预先备好的函数。但是有些库函数却要求应用先传给它一个函数，好在合适的时候调用，以完成目标任务。这个被传入的、后又被调用的函数就称为回调函数（callback function）。

例如：
有一家旅馆提供叫醒服务，但是要求旅客自己决定叫醒的方法。可以是打客房电话，也可以是派服务员去敲门，睡得死怕耽误事的，还可以要求往自己头上浇盆水。这里，“叫醒”这个行为是旅馆提供的，相当于库函数，但是叫醒的方式是由旅客决定并告诉旅馆的，也就是回调函数。而旅客告诉旅馆怎么叫醒自己的动作，也就是把回调函数传入库函数的动作，称为登记回调函数（to register a callback function）

可以看到，回调函数通常和应用处于同一抽象层（因为传入什么样的回调函数是在应用级别决定的）。而回调就成了一个高层调用底层，底层再回过头来调用高层的过程。（我认为）这应该是回调最早的应用之处，也是其得名如此的原因。



回调机制的优势
回调机制提供了非常大的灵活性。我们把图中的库函数改称为中间函数了，这是因为回调并不仅仅用在应用和库之间。任何时候，只要想获得类似于上面情况的灵活性，都可以利用回调。有的同学可定想回调似乎只是函数间的调用，可以发现两者之间的一个关键的不同：

在回调中，我们利用某种方式，把回调函数像参数一样传入中间函数。可以这么理解，在传入一个回调函数之前，中间函数是不完整的。
换句话说，程序可以在运行时，通过登记不同的回调函数，来决定、改变中间函数的行为。
这就比简单的函数调用要灵活太多了。


In [2]:
#回调函数1
#生成一个2k形式的偶数
def double(x):
    return x * 2

#回调函数2
#生成一个4k形式的偶数
def quadruple(x):
    return x * 4


#中间函数
#接受一个生成偶数的函数作为参数
#返回一个奇数
def getOddNumber(k, getEvenNumber):
    return 1 + getEvenNumber(k)

#起始函数，这里是程序的主函数
def main():
    k = 1
    #当需要生成一个2k+1形式的奇数时
    i = getOddNumber(k, double)
    print(i)
    #当需要一个4k+1形式的奇数时
    i = getOddNumber(k, quadruple)
    print(i)
    #当需要一个8k+1形式的奇数时
    i = getOddNumber(k, lambda x: x * 8)
    print(i)

if __name__ == "__main__":
    main()

3
5
9


#### 异步处理有关的回调函数

In [4]:
def apply_ascyn(func, args, callback):
    """
    func 函数的是处理的函数
    args 表示的参数
    callback 表示的函数处理完成后的 该执行的动作
    """
    result = func(*args)
    callback(result)

def add(x, y):
    return x + y

def print_result(result):
    print(result)

apply_ascyn(add, (2, 3), callback=print_result)
##这里print_result只能接收一个result的参数，不能传入其他信息。当想让回调函数访问其他变量或者特定环境的变量值的时候会遇到问题。

5


#### 使用一个绑定方法来代替这个简单函数

In [6]:
def appy_async(func, args, *, callback):
    result = func(*args)
    #异步执行的函数 同时将执行后返回到这个早从这个函数跳出去
    callback(result)

def add(x ,y):
    return x + y

##绑定方法的类
class ResultHandler(object):
    def __init__(self):
        self.sequence = 0

    def handle(self, result):
        self.sequence += 1
        print("[{}] Got: {}".format(self.sequence, result))

resultHandler = ResultHandler()
appy_async(add, (2,3), callback=resultHandler.handle)

[1] Got: 5


![jupyter](./img/20210912155646875.png)

#### 使用闭包函数

In [9]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def add(x ,y):
    return x + y

def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print("[{}] Got:{}".format(sequence, result))
    return handler

handler = make_handler()
apply_async(add, (2,3), callback=handler)

[1] Got:5


#### 协程

In [11]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def add(x, y):
    return x + y

def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print("[{}] Got:{}".format(sequence, result))

handle = make_handler()
next(handle)

apply_async(add, (2,3), callback=handle.send)

[1] Got:5
